# Uber-Data-Analysis-Project-in-Pyspark

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import max, window, avg, hour, sum, dayofweek, col, countDistinct, mean
from pyspark.sql.window import Window

In [2]:
# create a sparksession
spark = SparkSession.builder.appName("UberDataAnalysis").getOrCreate()

# Load the dataset into a dataframe
df = spark.read.csv('./data/uber.csv', header = True, inferSchema = True)

In [3]:
df.show(5)

+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     NULL|           8|        6|      0|               2|        2|            14|
|     NULL|           9|        8|      3|               0|        0|            14|
|     NULL|          10|        9|      2|               0|        1|            14|
|     NULL|          11|       11|      1|               4|        4|            11|
+---------+------------+---------+-------+----------------+---------+--------------+
only showing top 5 rows



In [4]:
print(df)

DataFrame[Date: string, Time (Local): int, Eyeballs : int, Zeroes : int, Completed Trips : int, Requests : int, Unique Drivers: int]


In [5]:
# Group the data and sum the completed trips
complete_trips_by_date = df.groupBy("Date").sum("Completed Trips ")
complete_trips_by_date.show()

+---------+---------------------+
|     Date|sum(Completed Trips )|
+---------+---------------------+
|11-Sep-12|                    1|
|13-Sep-12|                    0|
|17-Sep-12|                    0|
|18-Sep-12|                    3|
|21-Sep-12|                   10|
|     NULL|                 1261|
|15-Sep-12|                   23|
|12-Sep-12|                    0|
|19-Sep-12|                    0|
|14-Sep-12|                    3|
|16-Sep-12|                   17|
|23-Sep-12|                   23|
|24-Sep-12|                    1|
|22-Sep-12|                   19|
|20-Sep-12|                    2|
|10-Sep-12|                    2|
+---------+---------------------+



In [6]:
# Find the date with the most completed trips
date_with_completed_trips = \
    complete_trips_by_date.orderBy("sum(Completed Trips )", ascending = True).select("Date").first()['Date']

print(date_with_completed_trips)

13-Sep-12


In [7]:
# group the data by 24 hour windows and sum the completed trips
df = df.withColumn("Time (Local)", from_unixtime(col("Time (Local)")))

complete_trips_by_windows = df.groupBy(window("Time (Local)", "24 hours")).agg(
    sum("Completed Trips ").alias("Total Completed Trips")).orderBy("Total Completed Trips", ascending = False)

complete_trips_by_windows.show()

+--------------------+---------------------+
|              window|Total Completed Trips|
+--------------------+---------------------+
|{1970-01-01 08:00...|                 1365|
+--------------------+---------------------+



In [8]:
# Get the highest number of completed trips within a 24-hour period
highest_completed_trips_in_24_hours = complete_trips_by_windows.select("Total Completed Trips").first()[
    'Total Completed Trips']

print(highest_completed_trips_in_24_hours)

1365


In [9]:
hourly_requests = df.groupBy(hour("Time (Local)").alias("hour")).agg(sum("Requests ").alias("total_requests")).orderBy(
    "total_requests", ascending = False)

hourly_requests.show()

+----+--------------+
|hour|total_requests|
+----+--------------+
|   8|          1858|
+----+--------------+



In [10]:
most_requested_hour = hourly_requests.select("hour").first()[0]

print("The hour with the most requests is: ", most_requested_hour)

The hour with the most requests is:  8


In [11]:
print(df)

DataFrame[Date: string, Time (Local): string, Eyeballs : int, Zeroes : int, Completed Trips : int, Requests : int, Unique Drivers: int]


In [12]:
df.show(5)

+---------+-------------------+---------+-------+----------------+---------+--------------+
|     Date|       Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+-------------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|1970-01-01 08:00:07|        5|      0|               2|        2|             9|
|     NULL|1970-01-01 08:00:08|        6|      0|               2|        2|            14|
|     NULL|1970-01-01 08:00:09|        8|      3|               0|        0|            14|
|     NULL|1970-01-01 08:00:10|        9|      2|               0|        1|            14|
|     NULL|1970-01-01 08:00:11|       11|      1|               4|        4|            11|
+---------+-------------------+---------+-------+----------------+---------+--------------+
only showing top 5 rows



In [13]:
weekend_zeros = weekend_zeros = df.filter((hour("Time (Local)") >= 17) | (hour("Time (Local)") < 3)).filter(
    (dayofweek("Date") == 6) | (dayofweek("Date") == 7)).agg(sum("Zeroes ").alias("weekend_zeros")).collect()[0][
    "weekend_zeros"]

print(weekend_zeros)

None


In [14]:
# Calculate total zeros
total_zeros = df.agg(sum("Zeroes ").alias("total_zeros")).collect()[0]["total_zeros"]

print(total_zeros)

1429


In [15]:
weekend_zeros = 0
percent_weekend_zeros = weekend_zeros / total_zeros * 100

print("The percentage of zeros that occurred on weekends is: ", percent_weekend_zeros)

The percentage of zeros that occurred on weekends is:  0.0


In [16]:
print(df)

DataFrame[Date: string, Time (Local): string, Eyeballs : int, Zeroes : int, Completed Trips : int, Requests : int, Unique Drivers: int]


In [19]:
weighted_avg = df.withColumn("completed_per_driver", df["Completed Trips "] / df["Unique Drivers"]) \
    .groupBy("Date", "Time (Local)") \
    .agg(avg("completed_per_driver").alias("avg_completed_per_driver"),
         sum("Completed Trips ").alias("total_completed_trips")) \
    .withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips")) \
    .agg(sum("weighted_ratio") / sum("total_completed_trips")).collect()[0][0]

print("The weighted average ratio of completed trips per driver is:", weighted_avg)

The weighted average ratio of completed trips per driver is: 0.6008894493857285


In [23]:
# Calculate the number of unique requests for each hour of the day
hourly_unique_requests = (
    df.groupBy(hour("Time (Local)").alias("hour")).agg(countDistinct("Requests ").alias("unique_requests")))

hourly_unique_requests.show()

+----+---------------+
|hour|unique_requests|
+----+---------------+
|   8|             34|
+----+---------------+



In [30]:
# Side a window of 8 hours to find the busiest 8 consecutive hours
window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)

print(window)

In [31]:
busiest_8_consecutive_hours = (
    hourly_unique_requests
    .withColumn("consecutive_sum", sum("unique_requests").over(window))
    .orderBy(col("consecutive_sum").desc())
    .limit(1)
)

busiest_8_consecutive_hours.show()

+----+---------------+---------------+
|hour|unique_requests|consecutive_sum|
+----+---------------+---------------+
|   8|             34|             34|
+----+---------------+---------------+



In [35]:
# Group the data by 72 hour periods and calculate the radio of zeros to eyeballs for each period
period_ratios = (
    df.groupBy(((col("Date").cast("timestamp").cast("long") / (72 * 3600)).cast("int")).alias("period")).agg(
        sum("Zeroes ").alias("zeroes"), sum("Eyeballs ").alias("eyeballs")).withColumn("radio",
                                                                                       col("zeroes") / col("eyeballs")))

period_ratios.show()

+------+------+--------+-------------------+
|period|zeroes|eyeballs|              radio|
+------+------+--------+-------------------+
|  NULL|  1429|    6687|0.21369822042769554|
+------+------+--------+-------------------+



In [36]:
# Find the period with the highest radio
highest_raido_period = period_ratios.orderBy(col("radio").desc()).limit(1)

highest_raido_period.show()

+------+------+--------+-------------------+
|period|zeroes|eyeballs|              radio|
+------+------+--------+-------------------+
|  NULL|  1429|    6687|0.21369822042769554|
+------+------+--------+-------------------+



In [37]:
# Calculate requests per unique driver for each hour
requests_per_driver = (
    df.groupBy("Time (Local)").agg((sum("Requests ") / countDistinct("Unique Drivers")).alias("requests_per_driver")))

# show the hour with the highest radio
requests_per_driver.orderBy(desc("requests_per_driver")).show(1)

+-------------------+-------------------+
|       Time (Local)|requests_per_driver|
+-------------------+-------------------+
|1970-01-01 08:00:02|               20.0|
+-------------------+-------------------+
only showing top 1 row



In [41]:
# Calculate average completed trips and unique drivers for each hour
avg_trips_and_drivers = (df.groupBy('Time (Local)').agg(
    mean('Completed Trips ').alias('avg_completed_trips'),
    mean('Unique Drivers').alias('avg_unique_drivers')
))

# Show the hour with the lowest average completed trips and unique drivers
avg_trips_and_drivers.orderBy('avg_completed_trips', 'avg_unique_drivers').show(1)

+-------------------+-------------------+------------------+
|       Time (Local)|avg_completed_trips|avg_unique_drivers|
+-------------------+-------------------+------------------+
|1970-01-01 08:00:04|0.14285714285714285|0.6428571428571429|
+-------------------+-------------------+------------------+
only showing top 1 row

